In [1]:
%pip install transformers
%pip install mlflow
%pip install datasets   #!pip

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/98/46/f6a79f944d5c7763a9bc13b2aa6ac72daf43a6551f5fb03bccf0a9c2fec1/transformers-4.33.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/5e/5d/97afbafd9d584ff1b45fcb354a479a3609bd97f912f8f1f6c563cb1fae21/filelock-3.12.4-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/aa/f3/3fc97336a0e90516901befd4f500f08d691034d387406fdbde85bea827cc/huggingface_hub-0.17.3-py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.17 from https://files.pythonhosted.org/packages/b4/ca/fc1c4f8a2a4693ff437d039acf2dc93a190b9494569fbed246f535c44fc8/numpy-1.26.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/5

In [3]:
!pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 12.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.8 MB/s eta 0:00:00a 0:00:01


In [4]:
import torch
import mlflow
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import f1_score

In [5]:
# Load CIFAR-10 test split 
test_ds = load_dataset("cifar10", split="test")

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

/Users/travisrolle/travisrolle/lib/python3.9/site-packages/datasets/features/image.py:332: UserWarning: Downcasting array dtype uint8 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
mlflow.autolog()
with mlflow.start_run() as run:
  mlflow.log_params({
    "model_name": "vit-base-patch16-224-cifar10"  
    })
  mlflow.log_param("dataset_name", "CIFAR-10 Test")

  feature_extractor = ViTFeatureExtractor.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
  model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10')

  num_correct = 0
  true_labels = []
  predicted_labels = []
  for i, example in enumerate(test_ds):
    inputs = feature_extractor(images=example['img'], return_tensors="pt")
    outputs = model(**inputs)
    preds = outputs.logits.argmax(-1)

    true_labels.append(example['label'])
    predicted_labels.append(preds.item())

    if preds.item() == example['label']:
      num_correct += 1

  #calculate metrics
  accuracy = num_correct / len(test_ds) 
  f1 = f1_score(y_true=true_labels, y_pred=predicted_labels, average='macro')
  
  # Log metrics
  mlflow.log_metric("test_accuracy", accuracy)
  mlflow.log_metric("f1_score", f1) 

  # Log model
  mlflow.pytorch.log_model(model, "cifar10_model")

  print(run.info.artifact_uri)
  print(f"Run ID: {run.info.run_id}")

  #now run 'mlflow ui' in terminal to view it

  

2023/10/01 18:17:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2023/10/01 18:17:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2023/10/01 18:17:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/10/01 18:17:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
/Users/travisrolle/travisrolle/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


file:///Users/travisrolle/Downloads/mlruns/0/0750b522eb8b4cb5a1081e77c9bbe39c/artifacts
Run ID: 0750b522eb8b4cb5a1081e77c9bbe39c


In [8]:
mlflow.set_tracking_uri("http://192.168.5.172:5000")

In [9]:
import sys
sys.version


'3.9.6 (default, May  7 2023, 23:32:44) \n[Clang 14.0.3 (clang-1403.0.22.14.1)]'

In [10]:
import transformers
import torch

print("Transformers version:", transformers.__version__)
print("PyTorch version:", torch.__version__)



Transformers version: 4.33.3
PyTorch version: 2.0.1
